In [1]:
"""
Simple tester for the vgg19_trainable
"""
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf

import vgg16_new as vgg16
import os
import time
import numpy as np
from IPython.display import clear_output
import sys
import csv

PRETRAINED_MODEL_PATH= "/home/sik4hi/ckpt_dir"
N_EPOCHS = 300
INIT_LEARNING_RATE = 0.01
WEIGHT_DECAY_RATE = 0.0005
MOMENTUM = 0.9
IMAGE_HEIGHT  = 224    #960
IMAGE_WIDTH   = 224    #720
NUM_CHANNELS  = 3
BATCH_SIZE = 90
N_CLASSES = 1000
DROPOUT = 0.50
ckpt_dir = "/home/sik4hi/ckpt_dir"
LOGS_PATH = '/home/sik4hi/tensorflow_logs'
WEIGHT_PATH = '.npy'
TRAINSET_PATH = '/mnt/data1/imagenet-data/csv-files/train2/imagenetdataall0.csv'
VALSET_PATH0 ='/mnt/data1/imagenet-data/csv-files/val/imagenetdata0.csv'
VALSET_PATH1 ='/mnt/data1/imagenet-data/csv-files/val/imagenetdata1.csv'
VALSET_PATH2 ='/mnt/data1/imagenet-data/csv-files/val/imagenetdata2.csv'
VALSET_PATH3 ='/mnt/data1/imagenet-data/csv-files/val/imagenetdata3.csv'
VALSET_PATH4 ='/mnt/data1/imagenet-data/csv-files/val/imagenetdata4.csv'

os.environ["CUDA_VISIBLE_DEVICES"] = ""
#=======================================================================================================
# Reading Training data from CSV FILE
#=======================================================================================================
f=open(TRAINSET_PATH,"r")
filepaths=[]
labels=[]
for line in f:
    filepath, label= line.split(",")
    label= int(label)
    filepaths.append(filepath)
    labels.append(label)
    
with tf.device('/cpu:0'):
    def _compute_longer_edge(height, width, new_shorter_edge):
        return tf.cast(width*new_shorter_edge/height, tf.int32)

    
    one_ex = tf.train.slice_input_producer([filepaths,labels]
                                           ,capacity = 1281144, shuffle=True)
    
    
    im_content = tf.read_file(one_ex[0])
    train_image = tf.image.decode_jpeg(im_content, channels=3)
    
    # train_image = augment(train_image)
    #size = tf.cast([IMAGE_HEIGHT, IMAGE_WIDTH], tf.int32)
    #train_image = tf.image.resize_images(train_image, size)
    train_label = tf.cast(one_ex[1], tf.int64) # unnecessary
    
    
    shape = tf.shape(train_image)
    height = shape[0]
    width = shape[1]
    new_shorter_edge = tf.constant(256, dtype=tf.int32)

    height_smaller_than_width = tf.less_equal(height, width)
    new_height_and_width = tf.cond(
        height_smaller_than_width,
        lambda: (new_shorter_edge, _compute_longer_edge(height, width, new_shorter_edge)),
        lambda: (_compute_longer_edge(width, height, new_shorter_edge), new_shorter_edge)
    )
    size = tf.cast([new_height_and_width[0], new_height_and_width[1]], tf.int32)
    train_image = tf.image.resize_images(train_image, size)
    size = tf.cast([IMAGE_HEIGHT, IMAGE_WIDTH, 3], tf.int32)
    train_image = tf.random_crop(train_image, size)
    train_image = tf.image.random_flip_left_right(train_image)
    train_image = tf.cast(train_image, tf.float32)/255. # necessary for mapping rgb channels from 0-255 to 0-1 float.
    #train_label_batch = tf.one_hot(train_label_batch, 1000)
    train_image_batch, train_label_batch = tf.train.batch([train_image, train_label], batch_size=BATCH_SIZE,
                                                                  capacity = 24000 + 3*BATCH_SIZE,num_threads=24)


In [2]:
with tf.device('/cpu:0'):
    sess = tf.Session()
    init_op = tf.group(tf.initialize_all_variables(),
                       tf.initialize_local_variables())
    sess.run(init_op)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [3]:
try:
    epoch=0
    while not coord.should_stop() and epoch< N_EPOCHS:
        epoch+=1
        epoch_start_time = time.time()

        for i in range(1281144 / BATCH_SIZE):
            train_imbatch, train_labatch = sess.run([train_image_batch, train_label_batch]) 
            sys.stdout.write('\r\r' + "Iteration:" + str(i))
            sys.stdout.flush()

    
        print 'Time Elapsed for Epoch:' + str(epoch) + ' is ' + str(
            (time.time() - epoch_start_time) / 60.) + ' minutes'
except tf.errors.OutOfRangeError:
    print ("out of range")
finally:
    coord.request_stop()
    coord.join(threads)
    print("requesting stop")


Iteration:14233ation:1Time Elapsed for Epoch:2 is 44.0451971809 minutes
Iteration:14233Time Elapsed for Epoch:3 is 43.369676582 minutes
Iteration:257requesting stop


KeyboardInterrupt: 